<a href="https://colab.research.google.com/github/stunnedbud/CursoRedesProfundas/blob/main/Tarea2_Ejercicio1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarea 2: 1. Convolución

Para este ejercicio adaptaremos la red convolucional implementada en clase para que funcione sobre el conjunto Flowers102 de imagenes a color.

[Flowers102](https://www.robots.ox.ac.uk/~vgg/data/flowers/102/) tiene 102 classes teninendo entre 40 y 258 ejemplos por clase. La clasificación de flores es una tarea de grano fino difícil por las siguientes características.

* Baja variabilidad inter-clase: las diferencias entre las clases pueden ser sutiles.
* Baja variabilidad intra-clase: las diferencias entre los ejemplos de una misma clase pueden ser sutiles.

![Flowers102](https://www.researchgate.net/publication/345039740/figure/fig2/AS:1083904253534265@1635434320486/Oxford-102-Flower-dataset-which-contains-102-categories-of-flowers-can-be-used-for.jpg)


# 1. Conjunto de datos

### 1.1 Declarar bibliotecas



In [ ]:
from os.path import join
from urllib.request import urlopen

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
#import timm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as T
import torchvision.transforms.functional as TF
#import pytorch_lightning as pl
from IPython.display import display, HTML
from PIL import Image
from torch.utils.data import DataLoader
#from torchinfo import summary
from torchvision.datasets import Flowers102
from packaging import version


DATA_DIR = '../data'
LABELS_URL = 'https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/2023-1/data/flowers-102/labels.csv'
# for Kaggle, fix Flowers102 labels for torchvision < 0.13.0
FIX_LABELS = version.parse(torchvision.__version__) < version.parse('0.13.0')
target_transform = T.Lambda(lambda y: y - 1) if FIX_LABELS else nn.Identity()

## 1.2 Carga del conjunto de datos
Vamos a cargar el conjunto de imagenes de flores con la biblioteca que ofrece 'torchvision':

In [ ]:
#digits = load_digits() 
#zeros_ones = digits.target < 2
#data = digits.images[zeros_ones]
#labels = digits.target[zeros_ones]

trn_ds = Flowers102(DATA_DIR, 'train', target_transform=target_transform, download=True)
val_ds = Flowers102(DATA_DIR, 'val', target_transform=target_transform)
LABELS = pd.read_csv(LABELS_URL).labels

f'There are {len(trn_ds)} training and {len(val_ds)} validation examples'

Podemos visualizar una imagen del conjunto de entrenamiento:

In [ ]:
x, y = trn_ds[0]
npx = np.array(x)
print(f'Shape of single image: {npx.shape}')
print(f'Image PIL: {x}')
print(f'Mean of pixel values: {np.array(x).mean()}')
print(f'Label numeric: {y}, string: {LABELS[y]}')
x

### 1.3 Tuberia de datos

Un [tuberia de datos](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html) es la serie de pasos para leer y preprocesar los datos dejandolos listo para ser ingresar al modelo. Una tuberia tiene dos clases principales:

* [`Dataset`](https://pytorch.org/docs/stable/data.html#torch.utils.data.Dataset): carga un ejemplo y aplica una transformación.
* [`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader): agrupa los ejemplos y los barajea si es necesario (entrenamiento).

![Data Pipeline](https://raw.githubusercontent.com/bereml/riiaa-22-tl/master/figs/data_pipeline.svg)

In [ ]:
PRETRAIN_NORM = {
   'None': ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
   'ImageNet1k': ((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
   'ImageNet21k': ((0., 0., 0.), (1., 1., 1.))
}


def build_dl(subset, pretraining):
    """Builds a dataloader for the `subset` with the correct stats."""
    # get mean and standard deviation
    mean, std = PRETRAIN_NORM.get(pretraining, 'None')

    # assamble preprossing pipeline
    transform = T.Compose([
        # resize to small image
        T.Resize(256),
        # crop to te Center
        T.CenterCrop(224),
        # PIL to Tensor and [0-255] to [0, 1]
        T.ToTensor(),
        # normalize with stats
        T.Normalize(mean, std),
    ])

    # create the dataset
    dataset = Flowers102(DATA_DIR, subset, transform, target_transform)

    # create the dataloader to pack batches
    dataloader = DataLoader(
        dataset, 
        batch_size=32, 
        num_workers=4, 
        shuffle=subset=='train'
    )

    return dataloader

# creates a dataloader
val_dl = build_dl('val', 'None')

# loads the first batch
batch = next(iter(val_dl))
x, y = batch

# inspects batch
print(f'x.shape: {x.shape} x.mean: {x.mean()}')
print(f'y.shape: {y.shape} y[0]: {y[0]}')

## 2. Convolución y correlación cruzada

#### 2.1 En 1 canal
La operación de convolución entre una imagen en escala de grises $I$ y un filtro $W$ está definida por

$$
A_{i,j} = (\mathbf{I} * \mathbf{W})_{i,j} = \sum_m \sum_n I_{m, n} W_{i - m, j - n}
$$

La convolución es commutativa, por lo tanto 

$$
A_{i,j} = (\mathbf{W} * \mathbf{I})_{i,j} = \sum_m \sum_n I_{i - m, j - n} W_{m,n}
$$

En lugar de la convolución, frecuentemente se ocupa la operación de correlación cruzada para llevar a cabo las capas convolucionales. Esta operación es similar a la convolución pero sin voltear el filtro (por lo que pierde la propiedad de conmutatividad) y está dada por

$$
A_{i,j} = (\mathbf{W} * \mathbf{I})_{i,j} = \sum_m \sum_n I_{i + m, j + n} W_{m,n} 
$$


#### 2.2 En 3 canales

Para imagenes a color tenemos varias opciones de cómo aplicar filtros convolucionales. Podemos aplicar un filtro independiente por cada capa, resultando en 3 salidas o una salida con profundidad 3. O bien lo que haremos aquí es aplicar un filtro de tamaño 3 x 3 x 3 a las 3 capas al mismo tiempo para reducir a un sólo valor. 

La implementación en NumPy sería casi identica a la convolución en 2d, gracias a la sobrecarga del operador '*'. Solamente que ahora el filtro de entrada W y la imagen I deben ser de 3 dimensiones.




In [ ]:
def conv3d(I, W, b, stride = 1):
  h_s = int(np.floor((I.shape[0] - W.shape[0]) / stride)) + 1
  w_s = int(np.floor((I.shape[1] - W.shape[1]) / stride)) + 1
  a = np.zeros((h_s, w_s))
  for i in range(h_s):
    for j in range(w_s):
      I_m = I[i * stride:i * stride + W.shape[0], j * stride:j * stride + W.shape[1], :] # unico cambio necesario 
      a[i, j] = (I_m * W).sum() + b
                  
  return a

## Filtro

Podemos definir cualquier filtro de 3 dimensiones. Por ejemplo uno de tamaño $3 \times 3 \times 3$:

In [ ]:
# Activando la diagonal del filtro en 3d
filter1 = np.zeros((3,3,3))
np.fill_diagonal(filter1, np.array([1, 1, 1]))
print(filter1)

In [ ]:
# Activando las esquinas de cada capa
filter2 = np.array([[[1, 0, 1],[0, 0, 0],[1, 0, 1]],[[1, 0, 1],[0, 0, 0],[1, 0, 1]],[[1, 0, 1],[0, 0, 0],[1, 0, 1]]])
# Convolucionando todo en un parche de 3x3x3
filter3 = np.ones((3,3,3))
# Solo la esquina superior izq
filter4 = np.zeros((3,3,3))
filter4[0,0,0] = 1

filters = np.array([filter1, filter2, filter3, filter4])

Aplicando las operaciones de correlación cruzada y convolución de una imagen del dígito $0$ con dicho filtro y al revés obtenemos:

In [ ]:
x, y = trn_ds[0]
npx = np.asarray(x)
a1 = conv3d(npx, filter1, 3)
a2 = conv3d(npx, filter2, 5)
a3 = conv3d(npx, filter3, 1)
a4 = conv3d(npx, filter4, 2)
fig, axs = plt.subplots(1, 5, figsize=(10, 5))
axs[0].imshow(x, cmap = 'gray') 
axs[1].imshow(a1, cmap = 'gray') 
axs[2].imshow(a2, cmap = 'gray') 
axs[3].imshow(a3, cmap = 'gray')
axs[4].imshow(a4, cmap = 'gray')  
plt.show()

La imagen de la izquierda es la original, y las otras 4 son el mapa de activaciones en escala de grises, resultantes de aplicar cada filtro implementado. Podemos ver que son apenas ligeramente diferentes entre sí.

Podemos probar lo mismo con otra imagen:

In [ ]:
x, y = trn_ds[1]
npx = np.asarray(x)
a1 = conv3d(npx, filter1, 3)
a2 = conv3d(npx, filter2, 5)
a3 = conv3d(npx, filter3, 1)
a4 = conv3d(npx, filter4, 2)
fig, axs = plt.subplots(1, 5, figsize=(10, 5))
axs[0].imshow(x, cmap = 'gray') 
axs[1].imshow(a1, cmap = 'gray') 
axs[2].imshow(a2, cmap = 'gray') 
axs[3].imshow(a3, cmap = 'gray')
axs[4].imshow(a4, cmap = 'gray')  
plt.show()

Para aplicar la operación de convolución con cada uno de estos filtros definimos la siguiente función:

In [ ]:
def multi_conv3d(I, W, b, stride = 1):
  k = len(W)
  activations = []
  for i in range(k):
    activations.append(conv3d(I, W[i], b[i], stride = 1)) 
                       
  return np.array(activations)

Evaluamos esta función con cuatro distintos filtros para nuestras dos imágenes, obteniendo cuatro mapas de activaciones por cada imagen.

In [ ]:
x0, _ = trn_ds[0]
x1, _ = trn_ds[1]
npx0 = np.asarray(x0)
npx1 = np.asarray(x1)
b = [1,2,3,4] 
activations_0 = np.tanh(multi_conv3d(npx0, filters, b))
activations_1 = np.tanh(multi_conv3d(npx1, filters, b))

for i in range(4):
  plt.subplot(3,4,i + 1)
  plt.imshow(filters[i], cmap = 'gray')
  plt.subplot(3,4,i + 5)
  plt.imshow(activations_0[i], cmap = 'gray')
  plt.subplot(3,4,i + 9)
  plt.imshow(activations_1[i], cmap = 'gray')

Ahora definimos funciones para realizar un submuestreo máximo a un conjunto de mapas de características:

In [ ]:
def submuestreo_maximo(activations, block = (2,2)):
  H, W = activations.shape
  H_s = H // block[0]
  W_s = W // block[1]

  sub_a = np.zeros((H_s,W_s))
  max_x = activations.reshape((H * W_s, block[1])).max(axis = 1)
  sub_a = max_x.T.reshape((W_s, block[0], H_s)).max(axis = 1)
  
  return sub_a

def multi_submuestreo_maximo(activations, block = (2,2)):
  k = activations.shape[0]
  sub_a = []
  for i in range(k):
    sub_a.append(submuestreo_maximo(activations[i]))
                       
  return np.array(sub_a)

Aplicamos el submuestreo a nuestros mapas de activaciones

In [ ]:
sub_a0 = multi_submuestreo_maximo(activations_0)
sub_a1 = multi_submuestreo_maximo(activations_1)
for i in range(4):  

  plt.subplot(2,4,i + 1)
  plt.imshow(sub_a0[i], vmin = 0, cmap = 'gray')
  plt.subplot(2,4,i + 5)
  plt.imshow(sub_a1[i], vmin = 0, cmap = 'gray')